# Spark Walmart

## Import of libraries and modules

In [359]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import mean, min, max
from pyspark.sql.types import IntegerType,DateType,DoubleType


import pyspark.sql.functions as F


## Question 1 - Start a simple Spark Session

In [347]:
spark = SparkSession.builder\
                    .master("local[*]")\
                    .appName("walmart_stock")\
                    .getOrCreate()

## Question 2 - Load the Walmart Stock CSV File


In [348]:
data = spark.read\
          .option("header", True)\
          .csv(r"D:\Cours\M2\Spark-Hadoop\TD4\walmart_stock.csv")

# Showing top 5 rows
data.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



## Question 3 - What are the column names ?

In [349]:
columns_list = data.columns
print("The columns names are : ", columns_list)

The columns names are :  ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']


## Question 4 - What does the Schema look like ?

In [350]:
print("The schema looks like this : ")
data.printSchema()

#For the rest of the project, it is necessary to change the type of variables

data = data.withColumn("Date", data["Date"].cast(DateType()))\
           .withColumn("Open", data["Open"].cast(DoubleType()))\
           .withColumn("High", data["High"].cast(DoubleType()))\
           .withColumn("Low", data["Low"].cast(DoubleType()))\
           .withColumn("Close", data["Close"].cast(DoubleType()))\
           .withColumn("Volume", data["Volume"].cast(IntegerType()))\
           .withColumn("Adj Close", data["Adj Close"].cast(DoubleType()))

print("The new schema looks like this : ")
data.printSchema()

The schema looks like this : 
root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)

The new schema looks like this : 
root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### /!\ Required to make SQL queries


In [351]:
data.createOrReplaceTempView("data_SQL")

## Question 5 - Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day

In [352]:
# Python
CreateHV_ratio = data.withColumn("HV_ratio",col("High")/col("Volume"))
print(CreateHV_ratio.select('HV_ratio').show())


# SQL
spark.sql("""select High/Volume as HV_Ratio from data_SQL""").show()



+--------------------+
|            HV_ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows

None
+--------------------+
|            HV_Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.

## Question 6 - What day had the Peak High in Price ?

In [360]:
# Python
print(data.orderBy(col("High")\
          .desc())\
          .select(col("Date"))\
          .head(1))  

# SQL
spark.sql("""select Date from data_SQL order by High desc limit 1 """).show()

[Row(Date=datetime.date(2015, 1, 13))]
+----------+
|      Date|
+----------+
|2015-01-13|
+----------+



 ## Question 7 - What is the mean of the Close column ?

In [366]:
# Python
print("The mean of the Close column is :")
data.select(col("Close"))\
    .agg(avg(col("Close")).alias("Mean"))\
    .show()


# SQL
print("The mean of the Close column is :")
spark.sql("""select mean(Close) as Mean from data_SQL""").show()



The mean of the Close column is :
+-----------------+
|             Mean|
+-----------------+
|72.38844998012726|
+-----------------+

The mean of the Close column is :
+-----------------+
|             Mean|
+-----------------+
|72.38844998012726|
+-----------------+



## Question 8 - What is the max and min of the Volume column ?


In [365]:
# Python
data.select([min("Volume").alias("Minimum_volume"), max("Volume").alias("Maximum_volume")])\
    .show()


# SQL
spark.sql("""select min(Volume) as `Minimum_volume`, max(Volume) as `Maximum_volume` from data_SQL""").show()

+--------------+--------------+
|Minimum_volume|Maximum_volume|
+--------------+--------------+
|       2094900|      80898100|
+--------------+--------------+

+--------------+--------------+
|Minimum_volume|Maximum_volume|
+--------------+--------------+
|       2094900|      80898100|
+--------------+--------------+



## Question 9 - How many days was the Close lower than 60 dollars ?


In [364]:
# Python
data.filter(col("Close") < 60)\
    .agg(count(col("Date")).alias("DaysNumber"))\
    .show()


# SQL
spark.sql("""select count(Date) as DaysNumber from data_SQL where Close < 60""").show()

+----------+
|DaysNumber|
+----------+
|        81|
+----------+

+----------+
|DaysNumber|
+----------+
|        81|
+----------+



## Question 10 - What percentage of the time was the High greater than 80 dollars ?

In [300]:
# Python
perctime = data.filter(col("High")>80).count()/data.count()*100

print('There is {:0.2f}'.format(perctime),"%","of time which High was greater than $80.")


# SQL
spark.sql("""select round(((select count(High) from data_SQL where High > 80)/count(High)*100),2) as Percentage from data_SQL""").show()

There is 9.14 % of time which High was greater than $80.
+----------+
|Percentage|
+----------+
|      9.14|
+----------+



## Question 11 - What is the max High per year ?


In [376]:
# Python
data.groupby(year("Date").alias("Year"))\
    .agg(max(col("High")).alias("Max_High"))\
    .orderBy("Year")\
    .show() 

#SQL 
spark.sql("""select year(Date) as Year, max(High) as Max_High from data_SQL group by year order by year""").show()

+----+---------+
|Year| Max_High|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+

+----+---------+
|Year| Max_High|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



## Question 12 - What is the average Close for each Calendar Month ?

In [378]:
# Python
data.groupby(year("Date").alias("Year"),month("Date").alias("Month"))\
    .agg(avg(col("Close")).alias("Mean_Close"))\
    .orderBy("Year","Month")\
    .show()


# SQL
spark.sql("""select year(Date) as Year, month(Date) as Month, mean(Close) as Mean from data_SQL group by Year,Month order by Year,Month""").show()

+----+-----+------------------+
|Year|Month|        Mean_Close|
+----+-----+------------------+
|2012|    1|        60.2354999|
|2012|    2|            60.898|
|2012|    3|60.433636818181796|
|2012|    4|60.149000150000006|
|2012|    5|61.456363409090905|
|2012|    6| 67.50380961904762|
|2012|    7| 72.40666661904763|
|2012|    8| 73.04478265217392|
|2012|    9| 74.18157921052631|
|2012|   10| 75.30619061904761|
|2012|   11| 71.10952333333333|
|2012|   12| 69.71100009999999|
|2013|    1| 69.09476142857143|
|2013|    2| 70.62315857894738|
|2013|    3| 73.43649940000002|
|2013|    4| 77.68954572727273|
|2013|    5| 77.81636368181817|
|2013|    6| 74.97800020000001|
|2013|    7| 77.11545418181818|
|2013|    8| 75.22409204545455|
+----+-----+------------------+
only showing top 20 rows

+----+-----+------------------+
|Year|Month|              Mean|
+----+-----+------------------+
|2012|    1|        60.2354999|
|2012|    2|            60.898|
|2012|    3|60.433636818181796|
|2012|    4|60